<a href="https://colab.research.google.com/github/Sanchit9587/Pokemon_Hack2_Guild_App/blob/NER%26NLP/NER_Pokemon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U spacy
!python -m spacy download en_core_web_sm -q

import os
import random
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 130.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
TRAIN_DATA = [
    # --- Original style, fixed spellings ---
    (
        "In the heart of the forest, Pikachu darted between the trees, its cheeks sparking with electricity. Charizard soared overhead, casting a shadow on the ground below. Mewtwo watched silently from a distance, while Bulbasaur napped under a large leaf.",
        {"entities": [(32, 39, "POKEMON"), (98, 107, "POKEMON"), (147, 153, "POKEMON"), (183, 192, "POKEMON")]}
    ),
    (
        "Charizard and Pikachu teamed up to face Mewtwo in an epic battle. Bulbasaur cheered from the sidelines, its vines waving excitedly.",
        {"entities": [(0, 9, "POKEMON"), (14, 21, "POKEMON"), (40, 46, "POKEMON"), (72, 81, "POKEMON")]}
    ),
    (
        "Bulbasaur, Pikachu, and Charizard explored the ancient ruins, unaware that Mewtwo was observing their every move.",
        {"entities": [(0, 9, "POKEMON"), (11, 18, "POKEMON"), (24, 33, "POKEMON"), (69, 75, "POKEMON")]}
    ),
    (
        "Mewtwo challenged Charizard to a duel at sunrise. Pikachu and Bulbasaur watched with anticipation as the two powerful Pokémon clashed.",
        {"entities": [(0, 6, "POKEMON"), (18, 27, "POKEMON"), (41, 48, "POKEMON"), (53, 62, "POKEMON")]}
    ),
    (
        "Pikachu ran circles around Bulbasaur, while Charizard tried to keep up. Mewtwo simply levitated above them, amused by their antics.",
        {"entities": [(0, 7, "POKEMON"), (27, 36, "POKEMON"), (44, 53, "POKEMON"), (75, 81, "POKEMON")]}
    ),
    (
        "The four friends—Charizard, Pikachu, Mewtwo, and Bulbasaur—set out on a journey across the land, facing many challenges together.",
        {"entities": [(19, 28, "POKEMON"), (30, 37, "POKEMON"), (39, 45, "POKEMON"), (51, 60, "POKEMON")]}
    ),
    (
        "Charizard unleashed a fiery blast, but Pikachu dodged and countered with a thunderbolt. Mewtwo shielded Bulbasaur from the shockwave.",
        {"entities": [(0, 9, "POKEMON"), (45, 52, "POKEMON"), (83, 89, "POKEMON"), (99, 108, "POKEMON")]}
    ),
    (
        "Bulbasaur and Mewtwo worked together to solve the puzzle, while Pikachu and Charizard scouted ahead for danger.",
        {"entities": [(0, 9, "POKEMON"), (14, 20, "POKEMON"), (57, 64, "POKEMON"), (69, 78, "POKEMON")]}
    ),
    (
        "Pikachu, Bulbasaur, Charizard, and Mewtwo gathered around the campfire, sharing stories of their adventures.",
        {"entities": [(0, 7, "POKEMON"), (9, 18, "POKEMON"), (20, 29, "POKEMON"), (35, 41, "POKEMON")]}
    ),
    (
        "As the sun set, Mewtwo meditated quietly, while Charizard practiced its flying. Pikachu and Bulbasaur played nearby.",
        {"entities": [(16, 22, "POKEMON"), (45, 54, "POKEMON"), (68, 75, "POKEMON"), (80, 89, "POKEMON")]}
    ),

    # --- HQ-style new examples ---
    (
        "HQ confirms multiple Bulbasaurs nesting near the western perimeter. Pikachus have been spotted forming small groups, while Charizard circles overhead. Mewtwo remains unseen.",
        {"entities": [(20, 30, "POKEMON"), (62, 70, "POKEMON"), (112, 121, "POKEMON"), (129, 135, "POKEMON")]}
    ),
    (
        "Urgent: Pikachu interference disrupted comm lines last night. Charizard flew reconnaissance, Bulbasaur entrenched in tunnels, and Mewtwo projected illusions to confuse our scouts.",
        {"entities": [(8, 15, "POKEMON"), (54, 63, "POKEMON"), (65, 74, "POKEMON"), (100, 106, "POKEMON")]}
    ),
    (
        "Command reports: Mewtwo manipulating Bulbasaur swarms underground. Pikachu stationed near the power grid. Charizard’s shadow detected in the upper skies.",
        {"entities": [(17, 23, "POKEMON"), (38, 47, "POKEMON"), (68, 75, "POKEMON"), (101, 110, "POKEMON")]}
    ),
    (
        "Scouts intercepted Pikachus traveling east. Bulbasaur guarded the caves while Mewtwo observed from ruins. Charizard executed a flyover.",
        {"entities": [(19, 27, "POKEMON"), (39, 48, "POKEMON"), (63, 69, "POKEMON"), (98, 107, "POKEMON")]}
    ),
    (
        "HQ intel: Charizard hovering above the valley, while Pikachu clusters defended supply lines. Several Bulbasaurs detected underground. Mewtwo monitoring psychic signals.",
        {"entities": [(9, 18, "POKEMON"), (45, 52, "POKEMON"), (82, 92, "POKEMON"), (115, 121, "POKEMON")]}
    ),
    (
        "Field report: Bulbasaur activity near rivers increasing. Charizard spotted late evening. Pikachu remained neutral while Mewtwo vanished without trace.",
        {"entities": [(14, 23, "POKEMON"), (55, 64, "POKEMON"), (82, 89, "POKEMON"), (102, 108, "POKEMON")]}
    ),
    (
        "Satellite scan: Mewtwo coordinating Pikachu units. Bulbasaur roots blocking main roads. Charizard flying high to distract observers.",
        {"entities": [(16, 22, "POKEMON"), (36, 43, "POKEMON"), (45, 54, "POKEMON"), (83, 92, "POKEMON")]}
    ),
    (
        "Recon suggests Pikachu disrupting electronics in HQ. Charizard scouted ridges at dawn. Mewtwo hiding in nearby ruins, while Bulbasaurs advanced from marshes.",
        {"entities": [(14, 21, "POKEMON"), (52, 61, "POKEMON"), (71, 77, "POKEMON"), (107, 117, "POKEMON")]}
    ),
    (
        "Tactical briefing: Bulbasaur nests multiplying across lowlands. Charizard detected in high atmosphere. Pikachus forming swarms at nightfall. Mewtwo interference confirmed.",
        {"entities": [(19, 28, "POKEMON"), (58, 67, "POKEMON"), (96, 104, "POKEMON"), (124, 130, "POKEMON")]}
    ),
    (
        "Mission orders: Charizard to be monitored but not engaged. Mewtwo suspected behind psychic anomalies. Bulbasaur guarding caves, Pikachu maintaining neutrality.",
        {"entities": [(16, 25, "POKEMON"), (58, 64, "POKEMON"), (87, 96, "POKEMON"), (98, 105, "POKEMON")]}
    ),
    (
        "HQ alert: Pikachu units have established positions near generator fields. Bulbasaur vines detected in multiple sectors. Charizard patrolling skies, Mewtwo remains cloaked.",
        {"entities": [(9, 16, "POKEMON"), (70, 79, "POKEMON"), (111, 120, "POKEMON"), (128, 134, "POKEMON")]}
    ),
    (
        "Commander note: Several Bulbasaurs moving quickly near the ridge. Mewtwo observed briefly. Pikachu signals intercepted. Charizard remains above the horizon.",
        {"entities": [(19, 29, "POKEMON"), (63, 69, "POKEMON"), (81, 88, "POKEMON"), (118, 127, "POKEMON")]}
    ),
    (
        "HQ reports: Charizard engaged briefly but retreated. Bulbasaur blocking southern access. Pikachu scattered in the east. Mewtwo channeling psychic force.",
        {"entities": [(11, 20, "POKEMON"), (55, 64, "POKEMON"), (91, 98, "POKEMON"), (111, 117, "POKEMON")]}
    ),
    (
        "Intel confirms: Bulbasaurs multiplying rapidly. Pikachu groups avoiding confrontation. Mewtwo unseen for days. Charizard still circling mountain ranges.",
        {"entities": [(15, 25, "POKEMON"), (51, 58, "POKEMON"), (83, 89, "POKEMON"), (113, 122, "POKEMON")]}
    ),
    (
        "HQ directives: Neutralize Bulbasaur nests in sector four. Pikachus remain non-hostile. Charizard considered a high threat. Mewtwo controlling remotely.",
        {"entities": [(22, 31, "POKEMON"), (59, 67, "POKEMON"), (87, 96, "POKEMON"), (118, 124, "POKEMON")]}
    ),
    (
        "Patrols detected: Pikachu interacting with civilians. Bulbasaur entrenched near caves. Charizard sighted far above. Mewtwo gave no response.",
        {"entities": [(17, 24, "POKEMON"), (56, 65, "POKEMON"), (87, 96, "POKEMON"), (112, 118, "POKEMON")]}
    ),
    (
        "Recon logs: Charizard overflying ridge at dawn. Pikachu distracted scouts. Mewtwo spotted late evening. Bulbasaurs spread across riverbanks.",
        {"entities": [(11, 20, "POKEMON"), (46, 53, "POKEMON"), (63, 69, "POKEMON"), (97, 107, "POKEMON")]}
    ),
    (
        "HQ update: Bulbasaur presence high in marshes. Pikachu patrolling edges of supply lines. Charizard flying overhead. Mewtwo projecting mental images.",
        {"entities": [(10, 19, "POKEMON"), (45, 52, "POKEMON"), (82, 91, "POKEMON"), (99, 105, "POKEMON")]}
    ),
]


# ----------------------
# Settings: where to save the trained model
# ----------------------
# By default we save to /content/pokemon_ner (fast local). To persist across Colab restarts,
# set SAVE_TO_DRIVE = True and provide DRIVE_SAVE_PATH (see below).
MODEL_DIR = "/content/pokemon_ner"

# Always retrain when TRAIN_DATA changes
if os.path.exists(MODEL_DIR):
    # Remove the old model so retraining happens fresh
    import shutil
    shutil.rmtree(MODEL_DIR)

In [ ]:
print("Training new model...")
nlp = spacy.load("en_core_web_sm")
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner")
else:
    ner = nlp.get_pipe("ner")

LABEL = "POKEMON"
if LABEL not in ner.labels:
    ner.add_label(LABEL)

# Disable other pipes to only train NER
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.resume_training()
    n_iter = 40  # training iterations
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        batches = minibatch(TRAIN_DATA, size=compounding(2.0, 8.0, 1.5))
        for batch in batches:
            examples = []
            for text, ann in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, ann)
                examples.append(example)
            nlp.update(examples, sgd=optimizer, drop=0.35, losses=losses)
        print(f"Iter {itn+1}/{n_iter} — Losses: {losses}")

# Save trained model
nlp.to_disk(MODEL_DIR)
print("✅ Model saved to", MODEL_DIR)

Training new model...


/usr/local/lib/python3.12/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Charizard unleashed a fiery blast, but Pikachu dod..." with entities "[(0, 9, 'POKEMON'), (45, 52, 'POKEMON'), (83, 89, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Command reports: Mewtwo manipulating Bulbasaur swa..." with entities "[(17, 23, 'POKEMON'), (38, 47, 'POKEMON'), (68, 75...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be 

Iter 1/40 — Losses: {'ner': np.float32(77.63294)}
Iter 2/40 — Losses: {'ner': np.float32(71.764946)}
Iter 3/40 — Losses: {'ner': np.float32(64.15825)}
Iter 4/40 — Losses: {'ner': np.float32(56.88054)}
Iter 5/40 — Losses: {'ner': np.float32(50.080494)}
Iter 6/40 — Losses: {'ner': np.float32(45.126404)}
Iter 7/40 — Losses: {'ner': np.float32(37.947815)}
Iter 8/40 — Losses: {'ner': np.float32(28.20848)}
Iter 9/40 — Losses: {'ner': np.float32(26.754393)}
Iter 10/40 — Losses: {'ner': np.float32(23.124144)}
Iter 11/40 — Losses: {'ner': np.float32(20.778551)}
Iter 12/40 — Losses: {'ner': np.float32(15.714315)}
Iter 13/40 — Losses: {'ner': np.float32(12.010701)}
Iter 14/40 — Losses: {'ner': np.float32(9.841159)}
Iter 15/40 — Losses: {'ner': np.float32(5.46999)}
Iter 16/40 — Losses: {'ner': np.float32(7.9427304)}
Iter 17/40 — Losses: {'ner': np.float32(5.6683974)}
Iter 18/40 — Losses: {'ner': np.float32(3.4057071)}
Iter 19/40 — Losses: {'ner': np.float32(5.433141)}
Iter 20/40 — Losses: {'ner': 

In [ ]:
nlp = spacy.load(MODEL_DIR)

In [ ]:
def extract_pokemon_entities(text):
    """
    Returns list of dicts: [{"text": "Charizad", "label": "POKEMON"}, ...]
    """
    doc = nlp(text)
    return [{"text": ent.text, "label": ent.label_} for ent in doc.ents if ent.label_ == "POKEMON"]

In [ ]:
test_prompt = "Bulbasaur is peeing in the garden"
print("Input:", test_prompt)
print("NER output:", extract_pokemon_entities(test_prompt))

Input: Bulbasaur is peeing in the garden
NER output: [{'text': 'Bulbasaur', 'label': 'POKEMON'}]
